In [1]:
import duckdb
import os

In [2]:
con = duckdb.connect(database = "csecicids2018.duckdb", read_only = False)

In [3]:
con.execute('CREATE TABLE flows AS SELECT "Flow ID", Label FROM read_csv_auto("../datasets/CSECICIDS2018_improved/*.csv");')

In [4]:
con.execute("UPDATE Flows SET Label = regexp_replace(Label, '(.+) - Attempted','BENIGN')")

In [5]:
con.execute('SELECT DISTINCT Label FROM flows')
Labels = con.fetch_df()
Labels

,Label
0,DDoS-HOIC
1,Web Attack - XSS
2,Infiltration - NMAP Portscan
3,Web Attack - Brute Force
4,BENIGN
5,DDoS-LOIC-UDP
6,Botnet Ares
7,Infiltration - Communication Victim Attacker
8,DoS Slowloris
9,DoS GoldenEye


In [6]:
con.execute('SELECT Label, COUNT(*) AS count FROM Flows GROUP BY Label')
counted_labels = con.fetch_df()
counted_labels

,Label,count
0,DDoS-HOIC,1082293
1,Web Attack - XSS,113
2,Infiltration - NMAP Portscan,89374
3,Web Attack - Brute Force,131
4,BENIGN,59659723
5,Infiltration - Communication Victim Attacker,204
6,Botnet Ares,142921
7,DDoS-LOIC-UDP,2527
8,DoS Slowloris,8490
9,SSH-BruteForce,94197


In [7]:
con.execute('CREATE TABLE sampled_flows ("Flow ID" VARCHAR)')

In [8]:
con.execute('DELETE FROM sampled_flows')

for label in Labels.to_numpy():
    con.execute('INSERT INTO sampled_flows SELECT "Flow ID" FROM flows WHERE Label = \'{}\' ORDER BY RANDOM() LIMIT 2000'.format(label[0]))

In [9]:
con.execute('CREATE TABLE CSECICIDS2018_SAMPLED AS SELECT * EXCLUDE (id,"Flow ID","Src IP","Src Port","Dst IP","Dst Port",Timestamp) FROM read_csv_auto("../datasets/CSECICIDS2018_improved/*.csv") WHERE "Flow ID" IN (SELECT * FROM sampled_flows)')

In [10]:
con.execute("UPDATE CSECICIDS2018_SAMPLED SET Label = regexp_replace(Label, '(.+) - Attempted','BENIGN')")

In [11]:
con.execute("COPY (SELECT * FROM CSECICIDS2018_SAMPLED) TO '../datasets/CSECICIDS2018_improved.parquet' (FORMAT PARQUET)")